In [75]:
from openai import OpenAI
import os
import json
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [ ]:
#set up the env variables
client =OpenAI(
    api_key=os.getenv('RUNPOD_TOKEN'),
    base_url=os.getenv('RUNPOD_CHATBOT_URL'),

)
model_name= os.getenv("MODEL_NAME")

In [ ]:
#connect with runpod endpoints
def get_chat_response(client, model_name,messages,temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role":message["role"],"content":message["content"]})

    response = client.chat.completions.create(

        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=200
    ).choices[0].message.content

    return response

In [ ]:
#get LLM response
messages = [{"role":"system","content":"what is the capital of Germany"}]
response=get_chat_response(client,model_name,messages)

In [51]:
response

'The capital of Germany is Berlin.'

Prompt Engineering

Structured Output

In [52]:
#system prompt
system_prompt=""""
you are a helpful assistant that answer questions about capitals of countries.

Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object:
[{
    "country": the country that you will get the capital of
    "capital": the capital of the country stated
}]

"""
messages = [{"role":"system","content":system_prompt}]
messages.append({"role":"user","content":"what is the capital of Italy"})
response=get_chat_response(client,model_name,messages)
print(response)

[
    {
        "country": "Italy",
        "capital": "Rome"
    }
]


In [54]:
type(response)

str

In [ ]:
#json
json_response = json.loads(response)
json_response

[{'country': 'Italy', 'capital': 'Rome'}]

In [ ]:
#output which is convinient to be processed by other systems(helpful when coding the agents)
type(json_response[0]), json_response[0]['capital']

(dict, 'Rome')

In [60]:
##Input Structuring

In [61]:
user_input ="""
Get me capital of the following countries:
```
1. Italy
2. Germany
3. France
```
"""
messages = [{"role":"system","content":system_prompt}]
messages.append({"role":"user","content":user_input})
response=get_chat_response(client,model_name,messages)
print(response)


[
    {
        "country": "Italy",
        "capital": "Rome"
    },
    {
        "country": "Germany",
        "capital": "Berlin"
    },
    {
        "country": "France",
        "capital": "Paris"
    }
]


## Give the Model time to think(Chain of Thought)

In [62]:
user_prompt =""""
Calculate the result of this equation :1+3

Your output should be in a sctructured json format exactly like the one below.You are not allowed to write anything other than the json object:
{
    result: The final number resulted from calculating the equation above
}
"""
messages = [{"role":"user","content":user_prompt}]
response=get_chat_response(client,model_name,messages)
print(response)

{
    "result": 4
}


In [63]:
360/2*4567+35678*45-1000

2426570.0

In [64]:
user_prompt =""""
Calculate the result of this equation :360/2*4567+35678*45-1000

Your output should be in a sctructured json format exactly like the one below.You are not allowed to write anything other than the json object:
{
    result: The final number resulted from calculating the equation above
}
"""
messages = [{"role":"user","content":user_prompt}]
response=get_chat_response(client,model_name,messages)
print(response)

{
    "result": 814,851,800
}


In [65]:
#the above result is not accurate

In [66]:
user_prompt =""""
Calculate the result of this equation :360/2*4567+35678*45-1000

Your output should be in a sctructured json format exactly like the one below.You are not allowed to write anything other than the json object:
{
    steps: This is where you solve the equation bit by bit following the BEDMAS order of operations. You need to show your work and calculate each step leading to the final reult. Feel free to write in free text.
    result: The final number resulted from calculating the equation above
}
"""
messages = [{"role":"user","content":user_prompt}]
response=get_chat_response(client,model_name,messages)
print(response)

{
    "steps": "First, calculate the division: 360/2 = 180. Then, multiply 180 by 4567: 180*4567 = 818260. Next, multiply 35678 by 45: 35678*45 = 1606490. Now, add 818260 to 1606490: 818260 + 1606490 = 2467750. Finally, subtract 1000: 2467750 - 1000 = 2466750.",
    "result": 2466750
}


In [67]:
2466750 - 2426570.0

40180.0

## RAG - 
Helps the system to output information it doesn't have in its Memory

In [ ]:
#The Model was trained before Iphone 16 was released hence no info about Iphone 16
user_prompt = """"
what's new in iphone 16?

"""
messages = [{"role":"user","content":user_prompt}]
response=get_chat_response(client,model_name,messages)
print(response)


There is no such thing as an "iPhone 16". Apple has only released up to iPhone 14 series, and there is no official information about an iPhone 16.

However, if you're looking for information about the latest iPhone models or rumors about upcoming iPhone releases, I can provide you with some general information:

* The latest iPhone series is the iPhone 14 series, which includes the iPhone 14, iPhone 14 Plus, iPhone 14 Pro, and iPhone 14 Pro Max.
* The iPhone 14 series features improved cameras, faster processors, and new features like Crash Detection and Emergency SOS via Satellite.
* There have been rumors about upcoming iPhone releases, including the possibility of an iPhone 15 series with improved cameras, faster charging, and new design features. However, these are just rumors and not officially confirmed by Apple.

If you're looking for information about a specific iPhone model or feature, feel free to ask and I'll do my best to help!


In [72]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

In [73]:
user_prompt = f"""
{iphone_16}

What's new in iphone 16?
"""

messages = [{"role":"user","content":user_prompt}]
response=get_chat_response(client,model_name,messages)
print(response)

According to the text, the new features and updates in the iPhone 16 include:

1. Larger displays: 6.1-inch for the base model, 6.7-inch for the iPhone 16 Plus, 6.3 inches for the iPhone 16 Pro, and 6.9 inches for the iPhone 16 Pro Max.
2. Thinner bezels and a more durable Ceramic Shield.
3. New A18 chip (A18 Pro for the Pro models) with improved performance, neural engine capabilities, and faster GPU for gaming and machine learning tasks.
4. Upgraded camera systems:
	* Base iPhone 16: dual-camera setup with a 48MP main sensor.
	* Pro models: 48MP Ultra Wide and 5x telephoto camera, with Apple's "Camera Control" button for more flexible photography options.
5. Advanced audio features, including "Audio Mix" for refined audio capture during video recording.
6. Extended battery life, especially in


In [74]:
samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""

In [76]:
data =[iphone_16,samsung_s23]

In [77]:
user_prompt ="""
what is new in Iphone 16?
"""

In [78]:
embedding_client = OpenAI(
    api_key=os.getenv("RUNPOD_TOKEN"),
    base_url=os.getenv("RUNPOD_EMBEDDING_URL")
)

In [83]:
def get_embeddings(embedding_client, model_name,text_input):
    output=embedding_client.embeddings.create(input=text_input,model=model_name)

    embeddings = []
    for embedding_object in output.data:
        embeddings.append(embedding_object.embedding)
    return embeddings


In [87]:
user_prompt_embedding = get_embeddings(embedding_client,model_name,user_prompt)[0]

In [92]:
data_embeddings = [get_embeddings(embedding_client,model_name,text)[0] for text in data]

In [93]:
len(data_embeddings)

2

In [94]:
from sklearn.metrics.pairwise import cosine_similarity

In [95]:
data_similarity = cosine_similarity([user_prompt_embedding],data_embeddings)

In [96]:
data_similarity

array([[0.74480705, 0.52198179]])

In [97]:
closest_entry_index = data_similarity.argmax()
closest_entry_index

np.int64(0)

In [98]:
data[closest_entry_index]

'\nThe iPhone 16 introduces several exciting updates, making it one of Apple\'s most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.\n\nPowered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.\n\nApple also introduced advanced audio features like "Audio Mix," which uses machine l

In [99]:
user_prompt_with_data = f"""
    {data[closest_entry_index]}
{user_prompt}
"""

In [101]:
print(user_prompt_with_data)


    
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine lea

In [102]:

messages = [{"role":"user","content":user_prompt_with_data}]
response=get_chat_response(client,model_name,messages)
print(response)

According to the text, the new features and updates in the iPhone 16 include:

1. Larger displays:
	* Base model: 6.1-inch
	* iPhone 16 Plus: 6.7-inch
	* iPhone 16 Pro: 6.3-inch
	* iPhone 16 Pro Max: 6.9-inch
2. Thinner bezels and more durable Ceramic Shield
3. New A18 chip (A18 Pro for Pro models) with:
	* Enhanced neural engine capabilities
	* Faster GPU for gaming and machine learning tasks
4. Upgraded camera systems:
	* Base iPhone 16: Dual-camera setup with 48MP main sensor
	* Pro models: 48MP Ultra Wide and 5x telephoto camera
	* "Camera Control" button for more flexible photography options
5. Advanced audio features:
	* "Audio Mix" for separating background sounds from speech during video recording
6. Extended battery
